# Data Ingestion

In [30]:
from collections import namedtuple
import os


In [2]:
%pwd

'/Users/ravina/Desktop/IMAGECLASSIFICATION/research'

In [3]:
os.chdir("../")

In [10]:
%pwd

'/Users/ravina/Desktop/IMAGECLASSIFICATION'

In [47]:
os.chdir("/Users/ravina/Desktop/IMAGECLASSIFICATION")

In [31]:
%pwd

'/Users/ravina/Desktop/IMAGECLASSIFICATION/src'

In [31]:
DataIngestionConfig= namedtuple('DataIngestionConfig',[
    "root_dir",
    "source_URL",
    "local_data_file",
    "unzip_dir"
    ])

In [32]:
#entity: "return type" of a function
from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True)  #this is not python class but dataclass, here you can define the veriables without using self keyword.
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [33]:
## Configuration manager

from imageclassifier.constants import *
from imageclassifier.utils import read_yaml, create_directories
 


In [34]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        #schema_filepath = SCHEMA_FILE_PATH
        ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        #self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.get('data_ingestion')
        if config is None:
            raise ValueError("Data Ingestion configuration is missing or invalid.")

        create_directories([config.get('root_dir')])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [35]:
#components

import os 
import urllib.request as request
from zipfile import ZipFile
from imageclassifier import logger
from imageclassifier.utils.common import get_size

In [38]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            #logger.info(f"{filename} download! with following info: \n{headers}")
        #else:
            #logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def _get_updated_list_of_files(self,list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]
    
    def _preprocess(self,zf:ZipFile,f: str, working_dir: str):
        target_filepath=os.path.join(working_dir,f)
        if not os.path.exists(target_filepath):
            zf.extract(f,working_dir)
        
        if os.path.getsize(target_filepath)==0:
            os.remove(target_filepath)


    def unzip_and_clean(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with ZipFile(self.config.local_data_file, 'r') as zf:
            list_of_files=zf.namelist()
            updated_list_of_files=self._get_updated_list_of_files(list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf,f,self.config.unzip_dir)  

In [39]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

[2024-01-22 19:18:43,958: INFO: yaml file:config/config.yaml loaded successfully: common: yaml file:config/config.yaml loaded successfully]
[2024-01-22 19:18:43,960: INFO: yaml file:params.yaml loaded successfully: common: yaml file:params.yaml loaded successfully]
[2024-01-22 19:18:43,961: INFO: created directory at:artifacts: common: created directory at:artifacts]
[2024-01-22 19:18:43,962: INFO: created directory at:artifacts/data_ingestion: common: created directory at:artifacts/data_ingestion]


BadZipFile: File is not a zip file

In [55]:
%pwd

'/Users/ravina/Desktop/IMAGECLASSIFICATION'

[2024-01-22 19:01:09,619: INFO: yaml file:config/config.yaml loaded successfully: common: yaml file:config/config.yaml loaded successfully]
[2024-01-22 19:01:09,622: INFO: yaml file:params.yaml loaded successfully: common: yaml file:params.yaml loaded successfully]
[2024-01-22 19:01:09,623: INFO: created directory at:artifacts: common: created directory at:artifacts]
[2024-01-22 19:01:09,626: INFO: created directory at:artifacts/data_ingestion: common: created directory at:artifacts/data_ingestion]


TypeError: __init__() got an unexpected keyword argument 'root_dir'